In [53]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import pickle
import cv2
from skimage import img_as_float
from helpers import *
from ImageProcessing import *
from PIL import Image
from scipy import ndimage, signal

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Processing

In [54]:
# LOADING
root_dir = "training/"
image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) # Loading percentage p of images
imgs = [load_image(image_dir + files[i]) for i in range(n)]

# DATA MANIPULATION

# Image equalization
for i in range(n):
    uint8 = img_float_to_uint8(imgs[i])
    r,g,b = cv2.split(uint8)
    uint8_equalized = cv2.merge((cv2.equalizeHist(r), cv2.equalizeHist(g), cv2.equalizeHist(b)))
    imgs[i] = (img_as_float(uint8_equalized));

# Edge Enhancement

for i, img in enumerate(imgs):
    imgs[i] = cv2.resize((ChannelAugmentation([img]))[0][:,:,:3],(608,608))

# Neural Net

In [58]:
import tensorflow as tf
from tensorflow.python import keras

In [61]:
root_dir = "training/"
gt_dir = root_dir + "groundtruth/"
gt_imgs = np.array([cv2.resize(load_image(gt_dir + files[i]),(608,608)) for i in range(n)])
                      
gt_imgs[np.where(gt_imgs>=0.25)]=1
gt_imgs[np.where(gt_imgs<0.25)]=0

In [62]:
angles = [0, 90,180,270]
aug_imgs, aug_gt_imgs = DataAugmentation(imgs, gt_imgs, angles, True)

number_imgs = len(aug_imgs)

del imgs, gt_imgs

Augmenting the data with the images rotated by 0 deg.
Augmenting the data with the images rotated by 90 deg.
Augmenting the data with the images rotated by 180 deg.
Augmenting the data with the images rotated by 270 deg.
Augmenting the data with the symmetries


In [ ]:
x_train = keras.utils.normalize(np.array(aug_imgs))
del aug_imgs

y_train = keras.utils.to_categorical(aug_gt_imgs,num_classes=2)

del aug_gt_imgs
print("x_train shape:",x_train.shape)
print("y_train shape:", y_train.shape)

In [51]:
model = keras.Sequential()

model.add(keras.layers.Dense(100,activation='relu',input_shape=x_train.shape[1:4]))
model.add(keras.layers.Conv2D(filters=5,kernel_size=(2,2),strides=(2,2),padding='valid',data_format='channels_last',use_bias=True))
model.add(keras.layers.Conv2D(filters=5,kernel_size=(2,2),strides=(2,2),padding='valid',data_format='channels_last',use_bias=True))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Conv2DTranspose(filters=10,kernel_size=(2,2),strides=(2,2),activation='relu',padding='valid',data_format='channels_last',use_bias=True)),
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Conv2DTranspose(filters=5,kernel_size=(2,2),strides=(2,2)))

model.add(keras.layers.Dense(2,activation='softmax'))

model.summary()

NameError: name 'keras' is not defined

In [52]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

NameError: name 'model' is not defined

In [ ]:
model.fit(x_train,y_train,validation_split=0.2, epochs=50,verbose = 2)